<a href="https://colab.research.google.com/github/nyanta012/ERNIE-ViLG/blob/main/get_stock_price_tatibana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import datetime
import requests
import json
import time
import pytz
from tqdm import tqdm

In [2]:
tatibana_id = "****"
tatibana_pwd = "****"

In [ ]:
class TatibanaAPI:
    def __init__(self, tatibana_id, tatibana_pwd):
        params = {
            "p_no": "1",
            "p_sd_date": self.get_current_time().strftime("%Y.%m.%d-%H:%M:%S.%f")[
                :-3
            ],  # date+".000",
            "sCLMID": "CLMAuthLoginRequest",
            "sUserId": tatibana_id,
            "sPassword": tatibana_pwd,
            "sJsonOfmt": "4",
        }
        url = "https://kabuka.e-shiten.jp/e_api_v4r2/auth/"
        self.my_url = requests.get(url, params=json.dumps(params)).json()["sUrlRequest"]
        self.counter = 2
        self.tatibana_id = tatibana_id
        self.tatibana_pwd = tatibana_pwd

    def get_current_time(self):
        return datetime.datetime.now(pytz.timezone("Asia/Tokyo"))

    def get_price(self, code):
        params = {
            "p_no": str(self.counter),
            "p_sd_date": self.get_current_time().strftime("%Y.%m.%d-%H:%M:%S.%f")[
                :-3
            ],  # date+".000",
            "sCLMID": "CLMMfdsGetMarketPrice",
            "sTargetIssueCode": str(code),
            "sTargetColumn": "pDOP,pDHP,pDLP,pDPP,pDV,pDJ,pDYRP,xLISS",
            "sJsonOfmt": "4",
        }
        res = requests.get(self.my_url, params=json.dumps(params)).json()
        if res["p_errno"] == "0":
            self.counter += 1
        return res["aCLMMfdsMarketPrice"]

In [ ]:
runner = TatibanaAPI(tatibana_id=tatibana_id, tatibana_pwd=tatibana_pwd)

In [ ]:
code_list = [1332, 1333, 1414, 1417, 1605, 1721, 1801, 1802, 1803, 1808]

In [ ]:
data_list = []
for code in tqdm(code_list):
    data_list.extend(runner.get_price(int(code)))

In [ ]:
df = pd.DataFrame.from_dict(data_list)
rename_dict = {'sIssueCode':"コード",
               "pDOP":"始値",
               "pDHP":"高値",
               "pDLP":"安値",
               "pDPP":"現在値",
               "pDV":"出来高",
               "pDJ":"売買代金",
               "pDYRP":"騰落率",
               "xLISS":"市場",
              }
df = df.rename(columns=rename_dict)

In [ ]:
df